In [3]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from keras.preprocessing.text import Tokenizer, one_hot
from keras.optimizers import RMSprop
# from google.colab import drive
import numpy as np

In [4]:
# drive.mount('/content/gdrive/')


In [5]:
file_path = "shakespeare.txt"

with open(file_path,"r") as f:
  text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [6]:
print(text[31600:32000])

,
  Which though it alter not love's sole effect,
  Yet doth it steal sweet hours from love's delight.
  I may not evermore acknowledge thee,
  Lest my bewailed guilt should do thee shame,
  Nor thou with public kindness honour me,
  Unless thou take that honour from thy name:
    But do not so, I love thee in such sort,
    As thou being mine, mine is thy good report.


                     37
  


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [7]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 93


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [8]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 1821785


In [9]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
ration with World Library, Inc., from th

(Target Character): 
e


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (batch_size x seq_length x vocab_size).

In [10]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [11]:
X[0,:,:]

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [12]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [13]:
char_rnn = Sequential()
char_rnn.add(LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(Dense(len(chars),activation="softmax"))

In [14]:

char_rnn.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [16]:
char_rnn.fit(X,Y, epochs=1, batch_size=512)

Epoch 1/1
1821785/1821785 [==============================] - 856s 470us/step - loss: 1.4361


Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [17]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
      probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [18]:
sim = sample_text(char_rnn,sample_length=500) 

In [19]:
print(sim)

Ariadne and Antiopa?
  TITANIA. These arraning; whil the come in country'd upsorn'd him of Graf's tenradie,
    Subshion to mecom to no way's suctanies
    If then then no Nerdent to away those denchy
  and go.
  MAYMERSIE. And breaks hath curben'd, was but sun, while,
      By happy by womb?
  PROTEU. Leg how fathers consently,
    My save hath a goldhing theseived,
    Ay; and he way, and to tidon'd breeding on.
  GOSTRIDABURY. My brothers be discommardly and knight,
    Our 'twaef ourselves attend so that in wally?
  Beding. My ver


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.